# 4 Preprocessing and Training<a id='4_Preprocessing_and_Training'></a>

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Preprocessing and Training](#4_Preprocessing_and_Training)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load Data](#4.4_Load_Data)
  * [4.5 Data Cleaning](#4.5_Data_Cleaning)
      * [4.5.1 Drop columns from hold out set](#4.5.1_Drop_columns_from_hold_out_set)
      * [4.5.2 Missing Values](#4.5.2_Missing_Values)
      * [4.5.3 Drop Remaining Missing Values](#4.5.3_Drop_Remaining_Missing_Values)
      * [4.5.4 Outlier Removal](#4.5.4_Outlier_Removal)
  * [4.6 Split Dataset](#4.6_Split_Dataset)
  * [4.7 Preprocessing](#4.7_Preprocessing)
      * [4.7.1 Cadegorical Variable Cleaning](#4.7_Cadegorical_Variable_Cleaning)
      * [4.7.2 One-hot Encoding](#4.7.2_One-hot_Encoding)
      * [4.7.3 Train Test Split](#4.7.3_Train_Test_Split)
      * [4.7.4 Scale the Data](#4.7.4_Scale_the_Data)
  * [4.8 Save the Dataset](#4.8_Save_the_Dataset)

## 4.2 Introduction<a id='4.2_Introduction'></a>

In this notebook, I am going to first do some final data cleaning. Next, I will do preprocessing of categorical features and scaling of the data. Finally, I will break the training data into training and test splits to prepare for machine learning. 

## 4.3 Imports<a id='4.3_Imports'></a>

In [1]:
# import the necessary packages
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy import savetxt

## 4.4 Load Data<a id='4.4_Load_Data'></a>

In the previous notebook I saved the data as 'lc_step3.csv'. The hold-out dataset was saved in the data wrangling notebook as 'LC_2016_2017_cleaned.csv'. I will load this data now. 

In [2]:
df = pd.read_csv('lc_step3.csv', index_col = 'id')

In [3]:
df_hold = pd.read_csv('../data/LC_2016_2017_cleaned.csv', index_col='id')

## 4.5 Data Cleaning<a id='4.5_Data_Cleaning'></a>

Before preprocessing, I need to do some final data cleaning. I dropped a number of columns during EDA and now I am going to drop those from the hold-out set. Also, I did not 

### 4.5.1 Drop columns from hold out set<a id='4.5.1_Drop_columns_from_hold_out_set'></a>

In [4]:
# In the last notebook, I dropped a number of columns from the training set. Here I will drop those same columns
# from the hold-out set.
drop_col = ['installment', 'total_pymnt_inv', 'out_prncp_inv', 'recoveries', 'collection_recovery_fee', 'loan_status', 'emp_length', 'term(months)', 'grade']
df_hold.drop(drop_col, axis=1, inplace=True)
df_hold.shape

(759335, 34)

### 4.5.2 Missing Values<a id='4.5.2_Missing_Values'></a>

In [5]:
# training set missing values
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='%', ascending=False).head(6)

,count,%
next_pymnt_d,251160,28.375429
total_rev_hi_lim,68203,7.705404
tot_cur_bal,68203,7.705404
last_pymnt_d,17654,1.994505
last_credit_pull_d,53,0.005988
out_prncp,0,0.000000


In [6]:
missing_holdout = pd.concat([df_hold.isnull().sum(), 100 * df_hold.isnull().mean()], axis=1)
missing_holdout.columns=['count', '%']
missing_holdout.sort_values(by='%', ascending=False).head()

,count,%
next_pymnt_d,167915,22.113428
last_pymnt_d,948,0.124846
dti,355,0.046751
last_credit_pull_d,18,0.002370
total_pymnt,0,0.000000


There are still a number of missing values in the data that will need to be filled. However, should all of these columns be kept? 

If the goal is to create a model to determine whether a loan will or will not default, really we're trying to predict whether a loan should be approved or not. So, really, we should only use data that would be available before a loan is approved. 

Just in these missing values columns, I can see next_pymnt_d, tot_cur_bal, last_pymnt_d, and out_prncp that would not be available. What other columns should be removed?

In [7]:
df.columns

Index(['loan_amnt', 'int_rate', 'sub_grade', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'total_pymnt', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'defaults'],
      dtype='object')

It looks like 'issue_d', 'total_pymnt', 'total_rec_prncp','total_rec_int', 'total_rec_late_fee', 'last_pymnt_amnt', 'initial_list_status' would also not be available. I am going to drop these columns now.

In [8]:
dropCols = ['next_pymnt_d', 'tot_cur_bal', 'last_pymnt_d', 'out_prncp', 'issue_d', 'total_pymnt', 'total_rec_prncp','total_rec_int', 'total_rec_late_fee', 'last_pymnt_amnt', 'initial_list_status']
df.drop(dropCols, axis=1, inplace=True)
df_hold.drop(dropCols, axis=1, inplace=True)

Now which columns have missing data?

In [9]:
# training set missing values
missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='%', ascending=False).head()

,count,%
total_rev_hi_lim,68203,7.705404
last_credit_pull_d,53,0.005988
loan_amnt,0,0.000000
open_acc,0,0.000000
tot_coll_amt,0,0.000000


In [10]:
missing_holdout = pd.concat([df_hold.isnull().sum(), 100 * df_hold.isnull().mean()], axis=1)
missing_holdout.columns=['count', '%']
missing_holdout.sort_values(by='%', ascending=False).head()

,count,%
dti,355,0.046751
last_credit_pull_d,18,0.002370
loan_amnt,0,0.000000
open_acc,0,0.000000
total_rev_hi_lim,0,0.000000


### 4.5.3 Drop Remaining Missing Values<a id='4.5.3_Drop_Remaining_Missing_Values'></a>

In [11]:
df.dropna(inplace=True)
df_hold.dropna(inplace=True)

In [12]:
print(df.shape)
print(df_hold.shape)

(816880, 23)
(758962, 23)


In [13]:
df.isnull().sum()

loan_amnt                     0
int_rate                      0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
purpose                       0
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
last_credit_pull_d            0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
total_rev_hi_lim              0
defaults                      0
dtype: int64

In [14]:
df_hold.isnull().sum()

loan_amnt                     0
int_rate                      0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
purpose                       0
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
last_credit_pull_d            0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
total_rev_hi_lim              0
defaults                      0
dtype: int64

### 4.5.4 Outlier Removal<a id='4.5.4_Outlier_Removal'></a>

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_amnt,816880.0,14996.056428,8445.614217,1000.00,8400.00,13500.00,20000.00,35000.00
int_rate,816880.0,13.304700,4.407142,5.32,9.99,12.99,16.29,28.99
annual_inc,816880.0,75099.518403,48206.056157,0.00,46000.00,65000.00,90000.00,999999.00
dti,816880.0,18.487965,17.757974,0.00,12.19,17.97,24.37,9999.00
delinq_2yrs,816880.0,0.328371,0.883844,0.00,0.00,0.00,0.00,39.00
inq_last_6mths,816880.0,0.670748,0.958444,0.00,0.00,0.00,1.00,8.00
open_acc,816880.0,11.702545,5.353766,0.00,8.00,11.00,14.00,90.00
pub_rec,816880.0,0.208022,0.601369,0.00,0.00,0.00,0.00,86.00
revol_bal,816880.0,17126.539106,22367.763033,0.00,6591.75,12036.00,21078.00,2904836.00
revol_util,816880.0,55.317900,23.482086,0.00,38.10,56.00,73.50,892.30


The column 'dti' - A ratio calculated using the borrower's total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower's self:reported monthly income.

According to this definition, it would make sense to have a maximum value of 100. 

In [16]:
# defining a funcion to show the rows that appear to be outliers for a given column based on standard deviation
# input a column name and the number of std away from the mean (default is 3)
def std_outlier_detection(col, cut=3):
    mean = df[col].mean()
    std = df[col].std()
    cut_off = cut * std
    lower, upper = mean - cut_off, mean + cut_off
    return df[~((df[col]<upper) & (df[col]>lower))]

In [17]:
# defining a funcion to remove the rows that are above an upper limit for a given column
# input a column name and the upper limit
def outlier_removal_defined(col, upper):
    df = df[df[col]<upper]

In [18]:
# how many rows are outliers for dti?
std_outlier_detection('dti')

,loan_amnt,int_rate,sub_grade,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,...,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,total_rev_hi_lim,defaults
id,,,,,,,,,,,,,,,,,,,,,
68393780,25000.0,20.50,E4,MORTGAGE,19800.0,Verified,debt_consolidation,AL,83.64,0.0,...,0.0,11347.0,37.6,22.0,Jan-2016,0.0,0.0,2151.0,30200.0,0
67405134,6550.0,17.86,D5,MORTGAGE,1770.0,Not Verified,credit_card,TX,1092.52,2.0,...,0.0,29029.0,84.4,25.0,Jan-2016,0.0,0.0,0.0,34400.0,0
67565514,15400.0,14.48,C5,MORTGAGE,35000.0,Verified,debt_consolidation,NY,83.40,0.0,...,1.0,4424.0,63.2,38.0,Jan-2016,0.0,0.0,0.0,7000.0,0
67555887,15000.0,16.55,D2,MORTGAGE,20000.0,Source Verified,debt_consolidation,TX,137.40,0.0,...,1.0,21143.0,66.9,28.0,Jan-2016,0.0,0.0,0.0,31600.0,0
67575819,28000.0,19.99,E4,OWN,22246.8,Verified,debt_consolidation,CA,89.83,0.0,...,0.0,3527.0,92.8,20.0,Jan-2016,0.0,0.0,0.0,3800.0,0
67495417,28000.0,18.20,E1,RENT,17000.0,Source Verified,debt_consolidation,NV,136.97,0.0,...,1.0,9784.0,32.1,17.0,Jan-2016,0.0,0.0,0.0,30500.0,0
66415476,20000.0,16.99,D3,MORTGAGE,0.0,Not Verified,debt_consolidation,TX,9999.00,0.0,...,0.0,7161.0,88.4,11.0,Jan-2016,0.0,0.0,0.0,8100.0,0
66495833,20000.0,18.20,E1,RENT,27000.0,Source Verified,debt_consolidation,MD,100.09,0.0,...,0.0,30379.0,97.4,16.0,Jan-2016,0.0,0.0,0.0,31200.0,0
66593126,16000.0,17.57,D4,MORTGAGE,30000.0,Not Verified,debt_consolidation,GA,90.00,0.0,...,0.0,15225.0,41.5,39.0,Jan-2016,0.0,0.0,0.0,36700.0,0


Since there are not many dti outliers, I am just going to drop the rows. It looks like using std to drop rows will not be good for dti since rows with values that do make sense will be dropped. Instead I am just going to chose a fixed upper bound (of 100). 

In [19]:
# dropping rows where dti > 100
df = df[df['dti']<100]
df.shape

(816869, 23)

## 4.6 Split Dataset<a id='4.6_Split_Dataset'></a>

df currently contains both the features and the targets (the 'defaults' column) for machine learning. I will need to seperate these before moving forward. 

In [20]:
# Split the dataset into features and target
X = df.drop('defaults', axis=1)
y = df['defaults']

In [21]:
X_hold = df_hold.drop('defaults', axis=1)
y_hold = df_hold['defaults']

## 4.7 Preprocessing<a id='4.7_Preprocessing'></a>

### 4.7.1 Cadegorical Variable Cleaning<a id='4.7.1_Cadegorical_Variable_Cleaning'></a>

The categorical columns are currently dtype object. 

In [22]:
X.select_dtypes('object')

,sub_grade,home_ownership,verification_status,purpose,addr_state,earliest_cr_line,last_credit_pull_d
id,,,,,,,
10149342,B2,OWN,Verified,debt_consolidation,MI,Oct-1986,Dec-2015
10159584,C1,RENT,Not Verified,debt_consolidation,CA,Jan-2007,Jan-2016
10159498,A2,MORTGAGE,Not Verified,debt_consolidation,CT,Mar-1994,Jan-2016
10139658,B5,RENT,Source Verified,debt_consolidation,NM,Oct-1998,Jan-2016
10159548,A5,MORTGAGE,Not Verified,debt_consolidation,FL,Mar-1998,Jan-2016
...,...,...,...,...,...,...,...
36371250,B5,RENT,Verified,debt_consolidation,CA,Sep-2004,Jan-2016
36441262,B5,MORTGAGE,Verified,home_improvement,NJ,Mar-1974,Jan-2016
36271333,D2,RENT,Verified,debt_consolidation,TN,Sep-2003,Jan-2016


In [23]:
# Extracting year from the Earliest Credit Line and Last Credit Pulled columns
X['earliest_cr_line_year'] = X['earliest_cr_line'].str.slice(4, 8).astype('category')
X['last_credit_pull_d_year'] = X['last_credit_pull_d'].str.slice(4, 8).astype('category')

In [24]:
# drop the original date columns
X.drop(['earliest_cr_line', 'last_credit_pull_d'], axis=1, inplace=True)

In [25]:
# change the object columns to category
for col in ['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state']:
    df[col] = df[col].astype('category')

### 4.7.2 One-hot Encoding<a id='4.7.2_One-hot_Encoding'></a>

In [26]:
X = pd.get_dummies(X)

### 4.7.3 Train Test Split<a id='4.7.3_Train_Test_Split'></a>

Here I am going to split my dataset into training and test sets. The test set size is going to be 30%. Since only about 5% of loans default, I will need to set the stratify parameter.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

### 4.7.4 Scale the Data<a id='4.7.4_Scale_the_Data'></a>

For ML, I plan on using trees and logistic regression. These models do not assume that the data is a normal distribution. So, to scale the data, I am going to use a standard scaler. 

In [28]:
scale = StandardScaler()
scale.fit(X_train)
X_train_scaled = scale.transform(X_train)
X_test_scaled = scale.transform(X_test)

## 4.8 Save the Dataset<a id='4.8_Save_the_Dataset'></a>

In [29]:
# Save the data for use in the next modeling notebook
savetxt('X_train.csv', X_train_scaled, delimiter=',')
savetxt('X_test.csv', X_test_scaled, delimiter=',')
savetxt('y_train.csv', y_train, delimiter=',')
savetxt('y_test.csv', y_test, delimiter=',')

In [30]:
X_hold.to_csv('X_holdout.csv')
y_hold.to_csv('y_holdout.csv')